In [1]:
%matplotlib qt
from matplotlib import pyplot as plt
from numpy.random import rand
import skimage as sm
from skimage.io import imread, imsave
from skimage.util import img_as_float, random_noise
from skimage.transform import rotate
from skimage.filters import gaussian as ski_gaussian
from pylab import ginput
from scipy.signal import convolve, convolve2d, correlate2d
from scipy.signal import  gaussian as scipy_gaussian
from scipy.fft import fft2, fftshift
import numpy as np
import os
import timeit

os.chdir("../Mats")

In [ ]:
def plotGray(image): #Plots an image in gray scale
    plt.axis("off")
    plt.imshow(image,cmap="gray")

def plotRGB(image): #Plots an image in RGB color
    plt.axis("off")
    plt.imshow(image,cmap="jet")

# 1

1.1

In [ ]:
def gammaFuncGray(image,gamma):
    return 255*(image/255)**gamma

In [ ]:
def gammaFuncRGB(image,gamma):
    img = np.copy(image)
    for i in range(image.shape[2]):
        img[:,:,i] = 255*(img[:,:,i]/255)**gamma
    return img

In [2]:
def gammaFunc(image,gamma,HSV=False):
    img = np.copy(image)
    
    try:
        for i in range(image.shape[2]):
            img[:,:,i] = 255*(img[:,:,i]/255)**gamma
        return img
    except:
        return 255*(image/255)**gamma

In [ ]:
A = imread("cameraman.tif")

B=gammaFunc(A,0.5)
C=gammaFunc(A,2)


plt.subplot(1,3,1)
plotGray(A)
plt.title("Original Image")
plt.subplot(1,3,2)
plotGray(B)
plt.title("Gamma = 0.5 Image")
plt.subplot(1,3,3)
plotGray(C)
plt.title("Gamma = 2 Image")
plt.tight_layout()
plt.show()

1.2

In [ ]:
A = imread("autumn.tif")
B=gammaFunc(A,0.5)
C=gammaFunc(A,2)

plt.subplot(1,3,1)
plotGray(A)
plt.title("Original Image")
plt.subplot(1,3,2)
plotGray(B)
plt.title("Gamma = 0.5 Image")
plt.subplot(1,3,3)
plotGray(C)
plt.title("Gamma = 2 Image")
plt.tight_layout()
plt.show()

1.3

In [ ]:
def RBGtoHSV(image):
    img = np.copy(img_as_float(image))
    HSV_M = np.zeros(img.shape)
    H = np.zeros((img.shape[0],img.shape[1]))
    S = np.zeros((img.shape[0],img.shape[1]))
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    theta = np.arccos(((2*R-B-G)/2) / np.sqrt(R**2+G**2+B**2 - R*B - R*G - G*B + 1e-12))
    mask = img[:,:,2]>img[:,:,1]
    H[mask] = 2*np.pi - theta[mask]
    H[~mask] = theta[~mask]
        
    V =  np.amax(img,axis=2)
    mask = V > 0
    S[mask] = (V[mask]-np.amin(img,axis=2)[mask])/V[mask]
    S[~mask] = 0
    HSV_M[:,:,0] = H
    HSV_M[:,:,1] = S
    HSV_M[:,:,2] = V
    return HSV_M


In [ ]:
def gammaHSV(image,gamma):
    img = sm.color.rgb2hsv(image)
    img[:,:,2] = (img[:,:,2])**gamma
    return sm.color.hsv2rgb(img)

In [ ]:
A = imread("autumn.tif")
B=gammaHSV(A,0.5)
C=gammaHSV(A,2)

plt.subplot(1,3,1)
plotGray(A)
plt.title("Original Image")
plt.subplot(1,3,2)
plotGray(B)
plt.title("Gamma = 0.5 Image")
plt.subplot(1,3,3)
plotGray(C)
plt.title("Gamma = 2 Image")
plt.tight_layout()
plt.show()

# 2

2.1

In [70]:
def median_filter_optimized(image, N):
    pad_size = (N) // 2
    padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size)), mode='symmetric')
    output = np.empty_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            kernel = padded_image[i:i+N, j:j+N].ravel()
            kernel.sort()
            median_index = (N*N - 1) // 2
            output[i, j] = kernel[median_index]

    return output

def mean_filter_optimized(image, N):
    kernel = np.ones((N, N)) / N ** 2
    return convolve2d(image, kernel, mode='same')


In [21]:
def median_filter(image, N):
    mir_image = np.pad(image, np.int32(np.floor(N/2)+1), mode="symmetric")
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i, j] = np.median(mir_image[i:i+N,j:j+N].flatten())
    return image

def mean_filter(image, N):
    kernel = np.ones(N)/N**2
    mir_image = np.pad(image, np.int32(np.floor(N/2)+1), mode="symmetric")
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image [i,j] = np.sum(mir_image[i:i+N,j:j+N]*kernel)
    return image
    
A = imread("eight.tif")
executions = 100
N_max = 25
A_snp = random_noise(A, "s&p")
A_gaus = random_noise(A)
plt.subplot(1, 3, 1), plt.imshow(A, vmin=0, vmax=255, cmap="gray"), plt.title("Original")
plt.subplot(1, 3, 2), plt.imshow(A_snp, cmap="gray"), plt.title("s&p")
plt.subplot(1, 3, 3), plt.imshow(A_gaus, cmap="gray"), plt.title("Gaussian")
plt.show()

In [41]:
N = 3
plt.subplot(3, 2, 1), plt.imshow(A_snp, vmin=0, vmax=1,cmap="gray"), plt.title("s&p")
plt.subplot(3, 2, 2), plt.imshow(A_gaus, vmin=0, vmax=1, cmap="gray"), plt.title("Gaussian")
snp_mean_3 = mean_filter_optimized(np.copy(A_snp), N)
snp_median_3 = median_filter_optimized(np.copy(A_snp), N)
gaus_mean_3 = mean_filter_optimized(np.copy(A_gaus), N)
gaus_median_3 = median_filter_optimized(np.copy(A_gaus), N)
snp_mean_5 = mean_filter_optimized(np.copy(A_snp), 5)
snp_median_5 = median_filter_optimized(np.copy(A_snp), 5)
gaus_mean_5 = mean_filter_optimized(np.copy(A_gaus), 5)
gaus_median_5 = median_filter_optimized(np.copy(A_gaus), 5)

plt.subplot(3, 2, 3), plt.imshow(snp_mean_3, vmin=0, vmax=1, cmap="gray"), plt.title("s&p mean 3")
plt.subplot(3, 2, 4), plt.imshow(gaus_mean_3, vmin=0, vmax=1, cmap="gray"), plt.title("Gaussian mean 3")
plt.subplot(3, 2, 5), plt.imshow(snp_mean_5, vmin=0, vmax=1, cmap="gray"), plt.title("s&p mean 5")
plt.subplot(3, 2, 6), plt.imshow(gaus_mean_5, vmin=0, vmax=1, cmap="gray"), plt.title("Gaussian mean 5")
plt.show()

In [42]:
plt.subplot(3, 2, 1), plt.imshow(A_snp, vmin=0, vmax=1,cmap="gray"), plt.title("s&p")
plt.subplot(3, 2, 2), plt.imshow(A_gaus, vmin=0, vmax=1, cmap="gray"), plt.title("Gaussian")

plt.subplot(3, 2, 3), plt.imshow(snp_median_3, vmin=0, vmax=1, cmap="gray"), plt.title("s&p median 3")
plt.subplot(3, 2, 4), plt.imshow(gaus_median_3, vmin=0, vmax=1, cmap="gray"), plt.title("Gaussian median 3")
plt.subplot(3, 2, 5), plt.imshow(snp_median_5, vmin=0, vmax=1, cmap="gray"), plt.title("s&p median 5")
plt.subplot(3, 2, 6), plt.imshow(gaus_median_5, vmin=0, vmax=1, cmap="gray"), plt.title("Gaussian median 5")
plt.show()

In [71]:
def mean_100(image, N):
        for j in range(100):
            mean_filter_optimized(image, N)
        return

def median_100(image, N):
        for j in range(100):
           median_filter_optimized(image, N)
        return
  
mean_time = []
median_time = []
for i in range(1, N_max+1):
    print(i)
    start_time = timeit.default_timer()
    mean_100(np.copy(A), i)
    mean_time.append(timeit.default_timer()-start_time)
    start_time = timeit.default_timer()
    median_100(np.copy(A), i)
    median_time.append(timeit.default_timer()-start_time)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


ValueError: x and y must have same first dimension, but have shapes (1, 25) and (25,)

In [72]:
np.save("mean_time_data.npy", np.array(mean_time))
np.save("median_time_data.npy", np.array(median_time))


In [74]:
x_axis = np.arange(1, 26)
plt.plot(x_axis, mean_time, label="Mean time")
plt.plot(x_axis, median_time, label="Median time")

2.2

In [ ]:
def gaussian_filter(image, sigma, N):
    return
sigma = 5

2.3

# 3

In [ ]:
# Exercise 3 has been completed in the overleaf, see https://www.overleaf.com/6782353343wcfmggcfzyyd

# 4

4.1

In [7]:
A = imread("trui.png")

ps_A = fft2(A)
shift = fftshift(ps_A, )
plt.subplot(1, 3, 1), plt.imshow(A, cmap="gray"), plt.title("Original")
plt.subplot(1, 3, 2), plt.imshow(np.log10(1+np.abs(ps_A)**2), cmap="gray"), plt.title("fft2")
plt.subplot(1, 3, 3), plt.imshow(np.log10(1+np.abs(shift)**2), cmap="gray"), plt.title("fftshift")
plt.show()